In [1]:
import tensorflow as tf
import numpy as np
import os
import time

In [2]:
print(tf.__version__)

2.10.1


In [3]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
lst = []

with open('../data/full_tab_string.txt') as small_pf:

    tmp_list = []
    for line in small_pf:
        line = line.rstrip("\n")
        if line == "":
            lst.append(tmp_list)
            tmp_list = []
        else:
            tmp_list.extend(line.split())

    if tmp_list:  # add last one
        lst.append(tmp_list)
lst = lst[0]

In [5]:
text = ""
with open('../data/full_tab_string.txt') as small_pf:
    text = small_pf.read()

In [7]:
#Can't use set because tab encodings are not hashable types
def unique(list1):
    unique_set = set()
    for x in list1:
        if x not in unique_set:
            unique_set.add(x)
    return unique_set

In [8]:
#list of all tab locations
tab_vocab = unique(lst)

In [9]:
ids_from_chars = tf.keras.layers.StringLookup(
    num_oov_indices = 1, 
    vocabulary=list(tab_vocab), 
    mask_token=None)
print(ids_from_chars.get_vocabulary())

['[UNK]', 'e20', 'G17', 'B20', 'A75', 'e5', 'B10', 'D17', 'B15', 'e16', 'E11', 'B9', 'D10', 'e21', 'D9', 'G99', 'B22', 'B18', 'A33', 'e6', 'A14', 'D25', 'E77', 'e19', 'G14', 'D6', 'e2', 'G6', 'A6', 'e11', 'A5', 'D19', 'A11', 'B5', 'D77', 'G92', 'e12', 'e15', 'D0', 'E3', 'D12', 'D52', 'D5', 'E6', 'G19', 'B53', 'D3', 'B12', 'G4', 'E4', 'B16', 'A44', 'G91', 'E10', 'E2', 'E16', 'e18', 'E34', 'G23', 'E14', 'A12', 'e17', 'G21', 'E8', 'D16', 'e24', 'D15', 'G22', 'D14', 'G81', 'A45', 'A16', 'e3', 'D2', 'G16', 'E13', 'e7', 'B7', 'B51', 'B4', 'e77', 'B6', 'e9', 'E33', 'B8', 'B31', 'A13', 'e14', 'e10', 'A22', 'B17', 'B14', 'E57', 'B88', 'A10', 'D20', 'E9', 'A17', 'A1', 'E1', 'G11', 'B1', 'B11', 'G2', 'G5', 'e28', 'B21', 'E17', 'D22', 'G3', 'B33', 'A25', 'G24', 'E15', 'G7', 'B19', 'B13', 'G40', 'A19', 'D13', 'E5', 'A52', 'D7', 'E12', 'e23', 'D1', 'B24', 'A0', 'G42', 'G61', 'G13', 'B0', 'D75', 'A3', 'A4', 'e8', 'A54', 'G12', 'B23', 'B2', 'e1', 'A77', 'e0', 'A15', 'G0', 'e71', 'e22', 'G15', 'e13', '

In [10]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)


In [11]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids) + " ", axis=-1)

In [12]:
all_ids = ids_from_chars(tf.strings.split(text))

In [13]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)
type(ids_dataset)

tensorflow.python.data.ops.dataset_ops.TensorSliceDataset

In [14]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

D7
G5
B5
e5
e7
D6
B5
G5
e7
e8


In [15]:
seq_length = 10

In [16]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)
print(type(sequences))
for seq in sequences.take(1):
  print(chars_from_ids(seq))
  print(type(seq))

<class 'tensorflow.python.data.ops.dataset_ops.BatchDataset'>
tf.Tensor([b'D7' b'G5' b'B5' b'e5' b'e7' b'D6' b'B5' b'G5' b'e7' b'e8' b'D5'], shape=(11,), dtype=string)
<class 'tensorflow.python.framework.ops.EagerTensor'>


In [17]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b'D7 G5 B5 e5 e7 D6 B5 G5 e7 e8 D5 '
b'B5 G5 e8 e2 D4 B3 G2 e2 e0 D3 B1 '
b'G2 B1 e0 B1 G2 B0 G0 A2 B1 G2 A0 '
b'B1 G2 A0 G2 A0 A8 A7 A0 D7 G5 B5 '
b'e7 D6 B5 G5 e7 e8 D5 B5 G5 e8 e2 '


In [18]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [19]:
dataset = sequences.map(split_input_target)
dataset



<MapDataset element_spec=(TensorSpec(shape=(10,), dtype=tf.int64, name=None), TensorSpec(shape=(10,), dtype=tf.int64, name=None))>

In [20]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'D7 G5 B5 e5 e7 D6 B5 G5 e7 e8 '
Target: b'G5 B5 e5 e7 D6 B5 G5 e7 e8 D5 '


In [21]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset element_spec=(TensorSpec(shape=(64, 10), dtype=tf.int64, name=None), TensorSpec(shape=(64, 10), dtype=tf.int64, name=None))>

In [26]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 2048

In [27]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)

    #adjust the dense units size                         
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [28]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [29]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 10, 173) # (batch_size, sequence_length, vocab_size)


In [30]:
model.summary()

Model: "my_model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     multiple                  44288     
                                                                 
 gru_1 (GRU)                 multiple                  14168064  
                                                                 
 dense_1 (Dense)             multiple                  354477    
                                                                 
Total params: 14,566,829
Trainable params: 14,566,829
Non-trainable params: 0
_________________________________________________________________


In [31]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [32]:
sampled_indices

array([143,  15,  91, 130,  72,  53,  80,   3, 103, 138], dtype=int64)

In [33]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'G0 D7 G4 D5 G4 B0 B3 B0 G4 G4 '

Next Char Predictions:
 b'A15 G99 B14 G13 e3 E10 e77 B20 G2 B23 '


In [34]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [35]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 10, 173)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(5.1524916, shape=(), dtype=float32)


In [36]:
model.compile(optimizer='adam', loss=loss)

In [37]:
# Directory where the checkpoints will be saved
checkpoint_dir = './tab_training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [38]:
EPOCHS = 1000

In [39]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/1000
248/248 [==============================] - 5s 15ms/step - loss: 3.0467
Epoch 2/1000
248/248 [==============================] - 4s 15ms/step - loss: 2.2575
Epoch 3/1000
248/248 [==============================] - 4s 15ms/step - loss: 1.8537
Epoch 4/1000
248/248 [==============================] - 4s 15ms/step - loss: 1.5133
Epoch 5/1000
248/248 [==============================] - 4s 15ms/step - loss: 1.2402
Epoch 6/1000
248/248 [==============================] - 4s 15ms/step - loss: 1.0417
Epoch 7/1000
248/248 [==============================] - 4s 15ms/step - loss: 0.9190
Epoch 8/1000
248/248 [==============================] - 4s 16ms/step - loss: 0.8467
Epoch 9/1000
248/248 [==============================] - 4s 15ms/step - loss: 0.8013
Epoch 10/1000
248/248 [==============================] - 4s 14ms/step - loss: 0.7726
Epoch 11/1000
248/248 [==============================] - 4s 14ms/step - loss: 0.7525
Epoch 12/1000
248/248 [==============================] - 4s 14ms/step - lo

KeyboardInterrupt: 

In [40]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, omit, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.

    #USE THIS TO CHANGE WHICH OUTPUTS ARE ALLOWED
    skip_ids = self.ids_from_chars(omit)[:, None]

    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  #@tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]

    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [41]:
from MidiToTabOptions import *
tab_midi_dict = generate_tab_midi_pairs()

{40: ['E0'], 41: ['E1'], 42: ['E2'], 43: ['E3'], 44: ['E4'], 45: ['E5', 'A0'], 46: ['E6', 'A1'], 47: ['E7', 'A2'], 48: ['E8', 'A3'], 49: ['E9', 'A4'], 50: ['E10', 'A5', 'D0'], 51: ['E11', 'A6', 'D1'], 52: ['E12', 'A7', 'D2'], 53: ['E13', 'A8', 'D3'], 54: ['E14', 'A9', 'D4'], 55: ['E15', 'A10', 'D5', 'G0'], 56: ['E16', 'A11', 'D6', 'G1'], 57: ['E17', 'A12', 'D7', 'G2'], 58: ['E18', 'A13', 'D8', 'G3'], 59: ['E19', 'A14', 'D9', 'G4', 'B0'], 60: ['E20', 'A15', 'D10', 'G5', 'B1'], 61: ['E21', 'A16', 'D11', 'G6', 'B2'], 62: ['E22', 'A17', 'D12', 'G7', 'B3'], 63: ['E23', 'A18', 'D13', 'G8', 'B4'], 64: ['E24', 'A19', 'D14', 'G9', 'B5', 'e0'], 65: ['A20', 'D15', 'G10', 'B6', 'e1'], 66: ['A21', 'D16', 'G11', 'B7', 'e2'], 67: ['A22', 'D17', 'G12', 'B8', 'e3'], 68: ['A23', 'D18', 'G13', 'B9', 'e4'], 69: ['A24', 'D19', 'G14', 'B10', 'e5'], 70: ['D20', 'G15', 'B11', 'e6'], 71: ['D21', 'G16', 'B12', 'e7'], 72: ['D22', 'G17', 'B13', 'e8'], 73: ['D23', 'G18', 'B14', 'e9'], 74: ['D24', 'G19', 'B15', 'e1

In [42]:
#Array of Midi notes
notes = [60, 61 , 63, 60, 56, 58]
#notes = [74, 69, 67, 79, 69, 78, 69, 62, 74, 69, 67, 79, 69, 78, 69]
states = None

#First tab note, hard coded
base_tab = 'D7 '
#base_tab = 'D12 '


#Dynamically starting input 
append_tab = tf.constant([base_tab])
for note in notes:

    #Finds all tab options for the current note and 
    #removes them from the omitted option list in the OneStep function
    options = ['[UNK]']
    options.extend(tab_midi_dict[note])
    omit = list(set(ids_from_chars.get_vocabulary()) - set(options))
    omit.insert(0, '[UNK]')

    omit_sorted = []
    for item in ids_from_chars.get_vocabulary():
        if item in omit:
            omit_sorted.append(item)

    #generates the next predicted tab note out of the set of possible options
    one_step_model = OneStep(model, chars_from_ids, ids_from_chars, omit_sorted)

    #bootstrap the predictions 
    bootstrap_list = []    
    for i in range(100):
        next_char, states = one_step_model.generate_one_step(append_tab, states=states)    
        bootstrap_list.append(next_char.numpy()[0].decode('utf-8'))
    next_tab = max(set(bootstrap_list), key=bootstrap_list.count)


    #append new tab to input list
    append_tab = tf.constant([append_tab.numpy()[0].decode('utf-8') + next_tab + ' '])
    print(append_tab)

tf.Tensor([b'D7 G5 '], shape=(1,), dtype=string)
tf.Tensor([b'D7 G5 G6 '], shape=(1,), dtype=string)
tf.Tensor([b'D7 G5 G6 G8 '], shape=(1,), dtype=string)
tf.Tensor([b'D7 G5 G6 G8 G5 '], shape=(1,), dtype=string)
tf.Tensor([b'D7 G5 G6 G8 G5 D6 '], shape=(1,), dtype=string)
tf.Tensor([b'D7 G5 G6 G8 G5 D6 G3 '], shape=(1,), dtype=string)
